# Predicting CT Slice Locations
---
The primary purpose of this project is to assess the ability to use a matrix-based computational environment in the context of machine learning methods. In this project, `Python+NumPy` are mainly used, plus some functions in `SciPy`.

The dataset used in this project is from the UCI machine learning repository. Some features have been extracted from slices of CT medical scans, and the task is predicting the location of a slice in the body from features of the scan (regression task). 

Source of the dataset: https://archive.ics.uci.edu/ml/datasets/Relative+location+of+CT+slices+on+axial+axis

The patient IDs were removed from this version of the data, leaving 384 input features which were put in each of the “`X_...`” arrays. The corresponding CT scan slice location has been put in the “`y_...`” arrays. We shifted and scaled the “`y_...`” location values for the version of the data that you are using. The shift and scaling was chosen to make the training locations have zero mean and unit variance.

The first 73 patients were put in the `_train` arrays, the next 12 in the `_val` arrays, and the final 12 in the `_test` arrays.

## Getting Started
In this section, we load the dataset, take a glance, and do some data processing.

In [18]:
import numpy as np
data = np.load('ct_data.npz')
X_train = data['X_train']; X_val = data['X_val']; X_test = data['X_test']
y_train = data['y_train']; y_val = data['y_val']; y_test = data['y_test']

In [19]:
print('The dimension of X_train is', X_train.shape)
print('The dimension of X_val is', X_val.shape)
print('The dimension of X_test is', X_test.shape)

The dimension of X_train is (40754, 384)
The dimension of X_val is (5785, 384)
The dimension of X_test is (6961, 384)


In [20]:
print(f'The mean for y_val is {np.mean(y_val)}')
print(f'The sd for y_val is {np.sqrt(np.var(y_val))}')

The mean for y_val is -0.2160085093241599
The sd for y_val is 0.9814208579483531


In this part, we identify the features that take on the same value for every training example, and remove them in the three sets (training, validation and test set). Also, we identify identical columns in the training set and delete the later columns in the other three sets. 

In detail, we calculate the standard deviation of each column, and the column with 0 std is a constant feature. As for the detection of duplicate features, we sum up each column and identify the same numbers. 

In [21]:
# find the constant features
mask1 = np.std(X_train, 0) > 0
print('constant features: ', (mask1 != True).nonzero()[0])

# Take the sum of the rows. If matches to 32 bit precision, columns
# are probably the same. Yes, I could have been more careful here...
rc = np.array(np.dot(np.ones(X_train.shape[0]), X_train), dtype=np.float32)
_, idx = np.unique(rc, return_index=True)
mask2 = np.tile(False, X_train.shape[1])
mask2[idx] = True
print('duplicate features: ', (mask2 != True).nonzero()[0])
print('of which new to remove: ', ((mask2 != True) & mask1).nonzero()[0])

mask = mask1 & mask2
print('all features removing: ', (mask != True).nonzero()[0])

X_train = X_train[:, mask]
X_val = X_val[:, mask]
X_test = X_test[:, mask]
D = X_train.shape[1]

constant features:  [ 59  69 179 189 351]
duplicate features:  [ 69  78  79 179 188 189 199 287 351 359]
of which new to remove:  [ 78  79 188 199 287 359]
all features removing:  [ 59  69  78  79 179 188 189 199 287 351 359]


## Linear regression
Using `numpy.linalg.lstsq`, we write a short function `fit_linreg(X, yy, alpha)` that fits the linear regression model $$f(\textbf x;\textbf w,b) = \textbf w^{\top}\textbf x + b,$$ by minimizing the cost function: $$E(\textbf w, b) = \alpha\textbf w^{\top}\textbf w + \sum_n (f(\textbf x^{(n)};\textbf w,b) - y^{(n)})^2,$$ with regularization constant $\alpha$. Fitting a bias parameter $b$ and incorporating the regularization constant can both be achieved by augmenting the original data arrays. A quick revision, suppose we have $$\tilde{\textbf y} = \begin{bmatrix}\textbf y \\ \mathbf{0}_K\end{bmatrix} \quad \tilde{\Phi} = \begin{bmatrix}\Phi\\ \sqrt{\lambda}\mathbb I_K\end{bmatrix},$$ where $\mathbf{0}_K$ is a vector of $K$ zeros, and $\mathbb I_K$ is the $K\times K$ identity matrix. Then
$$\begin{align*}
    E(\textbf w;\,\tilde{\textbf y},\tilde{\Phi}) &= (\tilde{\textbf y} - \tilde{\Phi}\textbf w)^\top (\tilde{\textbf y} - \tilde{\Phi}\textbf w)\\
            &= (\textbf y - \Phi\textbf w)^\top (\textbf y - \Phi\textbf w) + \lambda\textbf w^\top\textbf w
            = E_\lambda(\textbf w;\,\textbf y,\Phi).
\end{align*}$$

Here, we use a data augmentation approach that maintains the numerical stability of the underlying `lstsq` solver, rather than a ‘normal equations’ approach. 

We use the constructed function to fit weights and a bias to `X_train` and `y_train`, with $\alpha=30$. We also fit the same model with a gradient-based optimizer with the help of `fit_linreg_gradopt` in the support code.

In [22]:
# the regularized linear regression function
def fit_linreg(X, yy, alpha):
    n = X.shape[0]
    # add a column of ones to X
    X = np.hstack([X, np.ones(n).reshape(n,1)])

    k = X.shape[1]
    # add the regulation to X
    X_tilde = np.vstack((X, np.sqrt(alpha)*np.eye(k)))

    yy = np.append(yy,[0]*k) # for the regulation
    return np.linalg.lstsq(X_tilde, yy, rcond=None)

# compute the RMSE
def rmse_linreg(fitmodel, X, yy):
    w = fitmodel[0] # coeficients
    n = X.shape[0]
    X = np.hstack([X, np.ones(n).reshape(n,1)])
    y_fit = np.dot(X, w)
    return np.sqrt(sum((y_fit-yy)**2)/n)

def rmse_gradopt(fitmodel, X, yy):
    w = fitmodel[0] # coeficients
    b = fitmodel[1]
    n = X.shape[0]
    y_fit = np.dot(X, w)+b
    return np.sqrt(sum((y_fit-yy)**2)/n)

In [23]:
from ct_support_code import *
fitmodel = fit_linreg(X_train, y_train, 30)
gradopt = fit_linreg_gradopt(X_train, y_train, 30)

Report the root-mean-square errors (RMSE) on the training and validation sets for the parameters fitted using both `fit_linreg` and the provided `fit_linreg_gradopt`.

In [24]:
print('For fit_linreg:')
print(f'The RMSE for training set is {rmse_linreg(fitmodel, X_train, y_train)}')
print(f'The RMSE for validation set is {rmse_linreg(fitmodel, X_val, y_val)}')
print('For fit_linreg_gradopt:')
print(f'The RMSE for training set is {rmse_gradopt(gradopt, X_train, y_train)}')
print(f'The RMSE for validation set is {rmse_gradopt(gradopt, X_val, y_val)}')

For fit_linreg:
The RMSE for training set is 0.35676698149487857
The RMSE for validation set is 0.42292954946321265
For fit_linreg_gradopt:
The RMSE for training set is 0.35675714280747745
The RMSE for validation set is 0.4230631666614895


The RMSE results are almost the same for the two models. They are similar because they both want to minimize the regularized least squares cost. However, `fit_linreg` finds the coeficients using matrix multiplication, `fit_linreg_gradopt` derives them by gradient descent, which result in the slight differences. The gradient solver may not have completely converged, but the least squares solver will only be accurate to numerical precision too. 

## Invented classification tasks
It is often easier to work with binary data than real-valued data: we don’t have to think so hard about how the values might be distributed, and how we might process them. Hence we invent some binary classification tasks, and fit these. 

We pick 20 positions within the range of training positions, and use each of these to define a classification task. 

The logistic regression cost function and gradients are provided in the function `logreg_cost`. It is analogous to the `linreg_cost` function for least-squares regression, which is used by the `fit_linreg_gradopt` function that we used earlier.

We fit logistic regression to each of the 20 classification tasks above with $\alpha=30$. Given a feature vector, we can now obtain 20 different probabilities, the predictions of the 20 logistic regression models. Then, we transform both the training and validation input matrices into new matrices with 20 columns, containing the probabilities from the 20 logistic regression models. 

Finally, we fit a regularized linear regression model ($\alpha=30$) to the transformed 20-dimensional training set, and report the training and validation root mean square errors (RMSE) of this model.

In [25]:
from scipy.optimize import minimize

# Fitting classifiers
alpha = 30
K = 20 # number of thresholded classification problems to fit
mx = np.max(y_train); mn = np.min(y_train); hh = (mx-mn)/(K+1)
thresholds = np.linspace(mn+hh, mx-hh, num=K, endpoint=True)
V_lr = np.zeros((K, D))
bb_lr = np.zeros(K)
for kk in range(K):
    print('Fitting classifier %d / %d' % (kk+1, K))
    labels = y_train > thresholds[kk]
    args = (X_train, labels, alpha)
    V_lr[kk,:], bb_lr[kk] = minimize_list(
            logreg_cost, (np.zeros(D), np.array(0)), args)

Fitting classifier 1 / 20
Fitting classifier 2 / 20
Fitting classifier 3 / 20
Fitting classifier 4 / 20
Fitting classifier 5 / 20
Fitting classifier 6 / 20
Fitting classifier 7 / 20
Fitting classifier 8 / 20
Fitting classifier 9 / 20
Fitting classifier 10 / 20
Fitting classifier 11 / 20
Fitting classifier 12 / 20
Fitting classifier 13 / 20
Fitting classifier 14 / 20
Fitting classifier 15 / 20
Fitting classifier 16 / 20
Fitting classifier 17 / 20
Fitting classifier 18 / 20
Fitting classifier 19 / 20
Fitting classifier 20 / 20


In [32]:
# Using least squares solver:
def fit_linreg(X, yy, alpha):
    # Least squares problem, regularizing weights but not bias:
    N, D = X.shape
    X_bias = np.hstack([np.ones((N,1)), X])
    X_reg = np.vstack([X_bias,
            np.hstack([np.zeros((D,1)), np.sqrt(alpha)*np.eye(D)])])
    yy_reg = np.hstack([yy, np.zeros(D)])
    params = np.linalg.lstsq(X_reg, yy_reg, rcond=None)[0]
    ww = params[1:]
    bb = params[0]
    return ww, bb

# Function to report root-mean-square-error (RMSE) to use later:
def rmse(y1,y2):
    return np.sqrt(np.mean((y1-y2)**2))

# Fitting dataset transformed using classifiers
P_fn = lambda X: 1/(1 + np.exp(-(np.dot(X,V_lr.T) + bb_lr[None,:])))
P_train = P_fn(X_train)
P_val = P_fn(X_val)
w_p, b_p = fit_linreg(P_train, y_train, alpha)
pred_p_train = np.dot(P_train, w_p) + b_p
pred_p_val = np.dot(P_val, w_p) + b_p
print('train_p_err = %g' % rmse(pred_p_train, y_train))
print('val_p_err = %g' % rmse(pred_p_val, y_val))

train_p_err = 0.154412
val_p_err = 0.254248


## Small neural network
In the last section we fitted a small neural network. The logistic regression classifiers are sigmoidal hidden units, and a linear output unit predicts the outputs. However, we didn’t fit the parameters jointly to the obvious least squares cost function. A least squares cost function and gradients for this neural network are implemented in the `nn_cost` function provided.

Here, we try fitting the neural network model to the training set, with a) a sensible random initialization of the parameters; b) the parameters initialized using the fits made in the last section.

In [40]:
from scipy.linalg import cho_factor, cho_solve

# a) Using random initialization of the parameters
def fit_nn_random(X, yy, alpha): 
    args = (X, yy, alpha)
    V = 0.1*np.random.randn(K, X.shape[1])/np.sqrt(X.shape[1])
    bk = 0.1*np.random.randn(K)/np.sqrt(K)
    ww = 0.1*np.random.randn(K)/np.sqrt(K)  
    bb = 0.1*np.random.randn(1)
    init = (ww, bb, V, bk)
    ww, bb, V, bk = minimize_list(nn_cost, init, args)
    return ww, bb, V, bk

# b) Using parameters initialized using the fits made in Q3
def fit_nn_logi(X, yy, alpha): 
    args = (X, yy, alpha)
    V = w_hat_m.T
    bk = b_hat_v
    ww = w_train  
    bb = b_train
    init = (ww, bb, V, bk)
    ww, bb, V, bk = minimize_list(nn_cost, init, args)
    return ww, bb, V, bk

# Prediction: 
n = X_train.shape[0]
np.random.seed(0)
ww, bb, V, bk = fit_nn_random(X = X_train, yy = y_train, alpha = 30)
para = (ww, bb, V, bk)
RMSE_random = np.sqrt(sum((nn_cost(para, X=X_train, yy=None, alpha=None) - y_train)**2)/n)
www, bbb, VV, bkk = fit_nn_logi(X = X_train, yy = y_train, alpha = 30)
para_logi = (www,bbb,VV,bkk)
RMSE_logi = np.sqrt(sum((nn_cost(para_logi, X=X_train, yy=None, alpha=None) - y_train)**2)/n)

print(RMSE_random)
print(RMSE_logi)

0.14058177895782742
0.13580955446486412


## Bayesian optimisation
A popular application area of Gaussian processes is Bayesian optimisation, where the uncertainty in the probabilistic model is used to guide the optimisation of a function. Here we will use Bayesian optimisation with Gaussian processes for choosing the regularisation parameter $\alpha$.

Gaussian processes are used to represent our belief about an unknown function. In this case, the function we are interested in is the neural network’s validation log root mean square error (log RMSE) as a function of the regularisation paramter $\alpha$. In Bayesian optimisation, it is common to maximise the unknown function, so we will maximise the negative log RMSE.

We start with a Gaussian process prior over this function. As we observe the actual log RMSEs for particular $\alpha$’s we update our belief about the function by calculating the Gaussian process posterior.

There are many popular acquisition functions in Bayesian optimisation. One example is the *probability of improvement*. Suppose we have observed $y^{(1)}$ to $y^{(n)}$ (here negative log RMSE at locations $\alpha^{(1)}$ to $\alpha^{(n)}$). Then the function takes the following form: $$\notag\mathit{PI}(\alpha) = \Phi\left(\frac{\mu(\alpha) - \text{max}(y^{(1)},\dots,y^{N})}{\sigma(\alpha)}\right),$$ 
where $\mu(\alpha)$ is the Gaussian process posterior mean at location $\alpha$, $\sigma(\alpha)$ is the posterior standard deviation at location $\alpha$, $\Phi$ denotes the cumulative density function of the Gaussian with mean 0 and variance 1. We pick the next regularization constant $\alpha^{(N+1)}$ by maximizing the acquisition function: $$\notag\alpha^{(N+1)} = \argmax_\alpha\mathit{PI}(\alpha).$$
We then evaluate our model for this regularization parameter and update our posterior about the unknown function that maps $\alpha$ to negative log RMSE. We repeat the procedure multiple times and then pick the parameter that yielded the best performance $y$.

In [16]:
import numpy as np
from ct_support_code import *
import scipy.stats

# trains the neural network for an alpha, returns the validation RMSE
def train_nn_reg(X, yy, X_val, y_val, alpha):
    ww, bb, V, bk = fit_nn_random(X, yy, alpha)
    para = (ww, bb, V, bk)
    y_fit = nn_cost(para, X=X_val)
    return np.sqrt(sum((y_fit-y_val)**2)/len(y_fit))

# function for generating y
base = train_nn_reg(X_train, y_train, X_val, y_val, 30)
def LRMSE(X, yy, X_val, y_val, alpha):
    return np.log(base/train_nn_reg(X, yy, X_val, y_val, alpha))

# select next alpha according to PoI
def next_alpha(mu_star, cov_star, y_alpha):
    PoI = scipy.stats.norm.pdf((mu_star - max(y_alpha))/np.sqrt(np.diag(cov_star)))
    max_PoI = max(PoI)
    max_alpha = X_star[np.where(PoI==max_PoI)[0][0]]
    return max_PoI, max_alpha

In [17]:
alphas = np.arange(0, 50, 0.02)
# Pick three alphas from this set for as training locations, 
ind = len(alphas)/4
X_alpha = alphas.take([ind-1,ind*2-1,ind*3-1])
# remaining locations as values for the acquisition function
X_star = np.setdiff1d(alphas, X_alpha)

# generate y for the picked alpha
y_alpha = np.zeros(len(X_alpha)) 
for i in range(len(X_alpha)):
    y_alpha[i] = LRMSE(X_train, y_train, X_val, y_val, X_alpha[i])


In [18]:
# retrain, do five iterations
iteration = 0
while iteration < 5:
    # calculate the posterior Gaussian Process
    mu_star, cov_star = gp_post_par(X_star, X_alpha, y_alpha)
    # select next alpha according to PoI
    max_PoI, alpha_new = next_alpha(mu_star, cov_star, y_alpha)
    iteration += 1
    print(f'The maximum probability of improvement is {max_PoI}, with alpha = {alpha_new}, for iteration {iteration}')
    X_alpha = np.append(X_alpha, alpha_new)
    X_star = np.setdiff1d(alphas, X_alpha)
    y_alpha = np.append(y_alpha, LRMSE(X_train, y_train, X_val, y_val, alpha_new))

The maximum probability of improvement is 0.3762439131308956, with alpha = 13.86, for iteration 1
The maximum probability of improvement is 0.3622776781757357, with alpha = 9.08, for iteration 2
The maximum probability of improvement is 0.34566890861340177, with alpha = 9.1, for iteration 3
The maximum probability of improvement is 0.35452121459600816, with alpha = 9.06, for iteration 4
The maximum probability of improvement is 0.3173822033001063, with alpha = 4.96, for iteration 5


In [19]:
# report the best alpha
best_alpha = X_alpha[np.where(y_alpha==max(y_alpha))[0][0]]

val_RMSE = train_nn_reg(X_train, y_train, X_val, y_val, best_alpha)
test_RMSE = train_nn_reg(X_train, y_train, X_test, y_test, best_alpha)

print(f'The best alpha is {best_alpha}, with its validation RMSE is {val_RMSE} and test RMSE is {test_RMSE}')

The best alpha is 12.48, with its validation RMSE is 0.2582302787839227 and test RMSE is 0.2892353771241661


The model is improved with smaller validation and test RMSEs. The observation noise can come from the random selection of initial parameters in the neural network. 

## What next
From the description of the data set, we know that the features indicate bone structures and air inclusions. In detail, after dropping the columns in question 1, the first to the 231th columns denotes bone structures. We know that for each CT slice (y), the bone structure should have a special pattern (or say, special shape). Hence, we are wondering if we can reshape the feature columns into a matrix, and employ CNN. However, this method involves theory about CNN, also relating to complicate coding, which we can’t apply given limited time, but we thought this may help.

In [42]:
import lightgbm as lgb

LGBR = lgb.LGBMRegressor() 
LGBR.fit(X_train, y_train)
y_pred = LGBR.predict(X_test)

rmse = np.sqrt(sum((y_pred-y_test)**2)/n)
rmse

0.1394614055917868